# Corporacion Favorita Grocery Sales Forecasting
# LSTM Sequential Prediction Pipeline

In [14]:
from datetime import date, timedelta
import gc 

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import keras
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras import callbacks
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


### Data loading and preprocessing

In [4]:
df_train = pd.read_csv(
    'train_oiled.csv', 
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  
)

df_test = pd.read_csv(
    "test_oiled.csv",
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)
df_test.columns = ['id', 'onpromotion', 'oil']

items = pd.read_csv(
    "data/items.csv",
).set_index("item_nbr")

# LSTM will train in a recent year data
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

"""
    Unstack time series data to represent each time series as a row
    with store-item combination as an index;
"""

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

oil_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["oil"]].unstack(
        level=-1).fillna(False)

oil_2017_train.columns = oil_2017_train.columns.get_level_values(1)
oil_2017_test = df_test[["oil"]].unstack(level=-1).fillna(0)
oil_2017_test.columns = oil_2017_test.columns.get_level_values(1)
oil_2017_test = oil_2017_test.reindex(oil_2017_train.index).fillna(0)
oil_2017 = pd.concat([oil_2017_train, oil_2017_test], axis=1)
del oil_2017_test, oil_2017_train
gc.collect()

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)
items = items.reindex(df_2017.index.get_level_values(1))

In [6]:
df_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   
          103520      0.693147    0.693147     ...        0.000000   
          103665      0.000000    2.079442     ...        0.693147   
          105574      1.386294    2.302585     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [7]:
"""
    Retrieve the data over the specified time period with a certain frequency
"""
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

"""
    Retrieve temporal features from preprocessed data ::
    
    Sales -      current value, mean values over n days, square mean values,
                 cumulative sales over n days, standard deviation in sales,
                 mean, std and squared mean for the last same n weekdays;
                 
    Promotions - same as for sales;
    
    Oil prices - mean, cumulative and deviation of oil prices over last n days;
"""

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "SQUARE_mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values ** 2,
        #"CUMULATIVE_3": get_timespan(df_2017, t2017, 3, 3).sum(axis=1).values,
        "std_3_2017": get_timespan(df_2017, t2017, 3, 3).std(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "CUMULATIVE_7": get_timespan(df_2017, t2017, 7, 7).sum(axis=1).values,
        "SQUARE_mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values ** 2,
        "std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values,
        "SQUARE_std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values ** 2,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "SQUARE_mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values ** 2,
        "std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values,
        "CUMULATIVE_14": get_timespan(df_2017, t2017, 14, 14).sum(axis=1).values,
        "SQUARE_std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values ** 2,
        
        "mean_21_2017": get_timespan(df_2017, t2017, 21, 21).mean(axis=1).values,
        "SQUARE_mean_21_2017": get_timespan(df_2017, t2017, 21, 21).mean(axis=1).values ** 2,
        "std_21_2017": get_timespan(df_2017, t2017, 21, 21).std(axis=1).values,
        #"CUMULATIVE_21": get_timespan(df_2017, t2017, 21, 21).sum(axis=1).values,
        #"SQUARE_std_21_2017": get_timespan(df_2017, t2017, 21, 21).std(axis=1).values ** 2,
        
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "SQUARE_mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values ** 2,
        "std_30_2017": get_timespan(df_2017, t2017, 30, 30).std(axis=1).values,
        "CUMULATIVE_30": get_timespan(df_2017, t2017, 30, 30).sum(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "CUMULATIVE_60": get_timespan(df_2017, t2017, 60, 60).sum(axis=1).values,
        "std_60_2017": get_timespan(df_2017, t2017, 60, 60).std(axis=1).values,
        "mean_90_2017": get_timespan(df_2017, t2017, 90, 90).mean(axis=1).values,
        "std_90_2017": get_timespan(df_2017, t2017, 90, 90).std(axis=1).values,
        #"mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        #"std_140_2017": get_timespan(df_2017, t2017, 140, 140).std(axis=1).values,
        
        ## PROMO _____________________________________________________________
        
        "promo_1_2017": get_timespan(promo_2017, t2017, 1, 1).values.ravel(),
        #"promo_3_2017": get_timespan(promo_2017, t2017, 3, 3).sum(axis=1).values,
        #"AVG_PROM_3": get_timespan(promo_2017, t2017, 3, 3).mean(axis=1).values,
        "proSTD_3_2017": get_timespan(promo_2017, t2017, 3, 3).std(axis=1).values,
        "promo_7_2017": get_timespan(promo_2017, t2017, 7, 7).sum(axis=1).values,
        "proSTD_7_2017": get_timespan(promo_2017, t2017, 7, 7).std(axis=1).values,
        "AVG_PROM_7": get_timespan(promo_2017, t2017, 7, 7).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        #"AVG_PROM_14": get_timespan(promo_2017, t2017, 14, 14).mean(axis=1).values,
        
        "proSTD_14_2017": get_timespan(promo_2017, t2017, 14, 14).std(axis=1).values,
        #"promo_21_2017": get_timespan(promo_2017, t2017, 21, 21).sum(axis=1).values,
        #"AVG_PROM_21": get_timespan(promo_2017, t2017, 21, 21).mean(axis=1).values,
        "proSTD_21_2017": get_timespan(promo_2017, t2017, 21, 21).std(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        #"AVG_PROM_60": get_timespan(promo_2017, t2017, 60, 60).mean(axis=1).values,
        #"promo_90_2017": get_timespan(promo_2017, t2017, 90, 90).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values,
        
        ## OIL _____________________________________________________________
        
        "NAFTA_30_2017": get_timespan(oil_2017, t2017, 21, 21).mean(axis=1).values,
        #"NAFTA_60_2017": get_timespan(oil_2017, t2017, 60, 60).mean(axis=1).values,
        #"NAFTA_90_2017": get_timespan(oil_2017, t2017, 90, 90).mean(axis=1).values,
        
        "CUM_NAFTA_30_2017": get_timespan(oil_2017, t2017, 21, 21).sum(axis=1).values,
        #"CUM_NAFTA_60_2017": get_timespan(oil_2017, t2017, 60, 60).sum(axis=1).values,
        #"CUM_NAFTA_90_2017": get_timespan(oil_2017, t2017, 90, 90).sum(axis=1).values,
        
        "STD_NAFTA_30_2017": get_timespan(oil_2017, t2017, 30, 30).std(axis=1).values,
        #"STD_NAFTA_60_2017": get_timespan(oil_2017, t2017, 60, 60).std(axis=1).values,
    })
    for i in range(7):
        X['promo_4_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['SQUARE_prAWmo_4_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values ** 2
        #X['SQUARE_prAWmo_12_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values ** 2
        #X['promo_12_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        #X['promo_20_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['mean_2_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').mean(axis=1).values
        #X['SQUARE_mean_2_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').mean(axis=1).values ** 2
        X['STD_2_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').std(axis=1).values
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['SQUARE_mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values ** 2
        #X['STD_4_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').std(axis=1).values
        #X['mean_8_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 56-i, 8, freq='7D').mean(axis=1).values
        #X['STD_8_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 56-i, 8, freq='7D').std(axis=1).values
        X['SQUARE_mean_8_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 56-i, 8, freq='7D').mean(axis=1).values ** 2
        X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        #X['STD_12_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').std(axis=1).values
        #X['SQUARE_mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values ** 2
        X['mean_16_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 112-i, 16, freq='7D').mean(axis=1).values
        #X['STD_16_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 112-i, 16, freq='7D').std(axis=1).values
        #X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        #X['STD_20_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').std(axis=1).values               
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [8]:
# Train set
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

# Validation and test sets
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

stores_items = pd.DataFrame(index=df_2017.index)
test_ids = df_test[['id']]

In [12]:
# LSTM-specific preprocessing
items = items.reindex( stores_items.index.get_level_values(1) )

X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
X_val = X_val.as_matrix()
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

## LSTM Model
### Tuned configuration

In [32]:
# Get the number of features to determine the number of nodes
nodes = X_train.shape[1]

model.add(LSTM(nodes, input_shape=(X_train.shape[1],X_train.shape[2]), 
               activation = 'tanh'))
model.add(Dropout(.12))
model.add(Dense(nodes, activation = 'relu'))
model.add(Dropout(.12))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'msle', optimizer='adamax', metrics=['msle'])

N_EPOCHS = 16

### Train the model

In [33]:
%%time
val_pred = []
test_pred = []

# Misprediction of perishable products induces more losses
sample_weights=np.array( pd.concat([items["perishable"]] * 6) * 0.25 + 1 )

for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    xv = X_val
    yv = y_val[:, i]
    model.fit(X_train, y, batch_size = 512, epochs = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv)) 
    val_pred.append(model.predict(X_val))
    test_pred.append(model.predict(X_test))

Step 1
Train on 1005090 samples, validate on 167515 samples
Epoch 1/25
 - 85s - loss: 0.1183 - mean_squared_logarithmic_error: 0.1122 - val_loss: 0.0951 - val_mean_squared_logarithmic_error: 0.0951
Epoch 2/25
 - 84s - loss: 0.1033 - mean_squared_logarithmic_error: 0.0980 - val_loss: 0.0962 - val_mean_squared_logarithmic_error: 0.0962
Epoch 3/25
 - 83s - loss: 0.1025 - mean_squared_logarithmic_error: 0.0972 - val_loss: 0.0972 - val_mean_squared_logarithmic_error: 0.0972
Epoch 4/25
 - 83s - loss: 0.1021 - mean_squared_logarithmic_error: 0.0969 - val_loss: 0.0943 - val_mean_squared_logarithmic_error: 0.0943
Epoch 5/25
 - 73s - loss: 0.1017 - mean_squared_logarithmic_error: 0.0966 - val_loss: 0.0936 - val_mean_squared_logarithmic_error: 0.0936
Epoch 6/25
 - 74s - loss: 0.1015 - mean_squared_logarithmic_error: 0.0964 - val_loss: 0.0937 - val_mean_squared_logarithmic_error: 0.0937
Epoch 7/25
 - 81s - loss: 0.1015 - mean_squared_logarithmic_error: 0.0964 - val_loss: 0.0950 - val_mean_squared_

KeyboardInterrupt: 

### Validate the prediction

In [12]:
n_public = 5 # Number of days in public test set
weights=pd.concat([items["perishable"]]) * 0.25 + 1
print("Unweighted validation mse: ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose()) )
print("Full validation mse:       ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose(), sample_weight=weights) )
print("'Public' validation mse:   ", mean_squared_error(
    y_val[:,:n_public], np.array(val_pred).squeeze(axis=2).transpose()[:,:n_public], 
    sample_weight=weights) )
print("'Private' validation mse:  ", mean_squared_error(
    y_val[:,n_public:], np.array(val_pred).squeeze(axis=2).transpose()[:,n_public:], 
    sample_weight=weights) )

Unweighted validation mse:  0.392228189502
Full validation mse:        0.391956139181
'Public' validation mse:    0.360790351786
'Private' validation mse:   0.406122406179


### Generate the predictions submission file

In [61]:
y_test = np.array(test_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_test, index=stores_items.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = test_ids.join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('LSTM.csv', float_format='%.5f', index=None)